# The exception mechanism

## What are exceptions

By now you have probably seen quite some errors in you code passing by: Syntax Errors, ValueErrors, IndexErrors etc. The first is something you cannot deal with in your program but the others are called (runtime) exceptions. They indicate that something exceptional has occurred. The Python interpreter is designed in such a way that when this happens you get a **_Traceback_** of the method calls all the way down the **_call stack_**. 
We will now investigate these concepts in more detail, and als introduce a mechanism that you can use to hook into the flow of exceptions.

Consider the code cell below, which has two methods calling each other to form a small call stack.

In [3]:
def process_data(data):
    print("processing data")
    for e in data:
        process_person(e)

def process_person(tup):
    print("processing person")
    print(f"last name={tup[1]}")

my_data = [("Mike", "Mutter"), ("Ralph Racker", ), ("Louis", "Levee")]
process_data(my_data)


processing data
processing person
last name=Mutter
processing person


IndexError: tuple index out of range

From the "global" context, which is the executing cell in this case (this is rather different when running scripts), method `process_data` is called and from this method `process_person`. everything is going hunky-dory until the `process_person` method tries to access a non-existing tuple element of the second person.  
The Python interpreter has a special kind of error to inform you of such events: the `IndexError`.

Because we have no error / exception event handling in place, the error "falls" all the way through the call stack until it reaches "main". The interpreter then exits execution with a representation of the route to the origin of the error.  
Fortunately this traceback is really informative; it gives you in nice highlighted text 

- The type of error with extra info: `IndexError: tuple index out of range`
- The state of the call stack at the moment the error occurred
- The origin of the error: `tup[1]`

![](pics/python_traceback.png)

## Catching exceptions

So are we completely helpless in case of such events? No! We have have the tools to hook into the exception mechanism to prevent a crashing program. The basic tool is the try/except block:

```python
try:
    # do something risky
except:
    # recover from error event
```


Given our apparently risky method:

```python
def process_person(tup):
    print("processing person")
    print(f"last name={tup[1]}")
```

We need to ask ourselves "Is there a erasonable way to recover from the event where a person has no last name?

In this case there is one, and it is simply assigning a deafult name in case of absence:



In [4]:
def process_person(tup):
    print("processing person")
    try:
        last_name = tup[1]
    except:
        last_name = "UNKNOWN"
    print(f"last name={last_name}")
    

Now when the same data processing is performed we simply have an unknwon person in our collection

In [6]:
my_data = [("Mike", "Mutter"), ("Ralph Racker", ), ("Louis", "Levee")]
process_data(my_data)

processing data
processing person
last name=Mutter
processing person
last name=UNKNOWN
processing person
last name=Levee


But wait! There is more. What if a completely different error occurs from the one we were expecting? Here is a hypothetical example"

In [10]:
def process_person(tup):
    print("processing person")
    try:
        last_name = tup[1]
        if last_name == "Levee":
            x = 1/0
    except:
        last_name = "UNKNOWN"
    print(f"last name={last_name}")

In [11]:
my_data = [("Mike", "Mutter"), ("Ralph", "Racker"), ("Louis", "Levee")]
process_data(my_data)

processing data
processing person
last name=Mutter
processing person
last name=Racker
processing person
last name=UNKNOWN


Now we have an unknown person when there actually is a last name for this person. This can be solved by catching the right type of exception.

In [12]:
def process_person(tup):
    print("processing person")
    try:
        last_name = tup[1]
        if last_name == "Levee":
            x = 1/0
    except IndexError:
        last_name = "UNKNOWN"
    print(f"last name={last_name}")

The consequence is that the name handling is correct, but the `ZeroDivisionError` will still cause the system to crash.

In [13]:
my_data = [("Mike", "Mutter"), ("Ralph", "Racker"), ("Louis", "Levee")]
process_data(my_data)

processing data
processing person
last name=Mutter
processing person
last name=Racker
processing person


ZeroDivisionError: division by zero

We can catch an exception where it occurs or anywhere "below" the risky operation in the call stack. Here is another solution where the person causing the error is completely ignored:

In [14]:
def process_data(data):
    print("processing data")
    for e in data:
        try:
            process_person(e)
        except IndexError:
            print("--PERSON IGNORED--")

def process_person(tup):
    print("processing person")
    print(f"last name={tup[1]}")

my_data = [("Mike", "Mutter"), ("Ralph Racker", ), ("Louis", "Levee")]
process_data(my_data)


processing data
processing person
last name=Mutter
processing person
--PERSON IGNORED--
processing person
last name=Levee


Finally, you can get a hold of the actual error object and extract some information from it:

In [18]:
def process_data(data):
    print("processing data")
    for e in data:
        process_person(e)

def process_person(tup):
    print("processing person")
    try:
        print(f"last name={tup[1]}")
    except IndexError as e:
        print(f"ERROR: {e} on {tup}")

my_data = [("Mike", "Mutter"), ("Ralph Racker", ), ("Louis", "Levee")]
process_data(my_data)


processing data
processing person
last name=Mutter
processing person
ERROR: tuple index out of range on ('Ralph Racker',)
processing person
last name=Levee


## Raising exceptions

Sometimes you want to message your own error, or "convert" a previously cought exception into another type. In that case you will want to `raise` an exception in your code yourself.

In [20]:
def raise_it():
    print("going for it")
    raise ValueError("chickening out")
    
raise_it()

going for it


ValueError: chickening out